In [2]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import seaborn as sns

In [3]:
df = pd.read_parquet(r"/Users/loufourneaux/Desktop/EPFL/MA1/ML/project2/All_Relative_Results_Cleaned.parquet")

In [4]:
df_clean = df.dropna()
index = df_clean.columns.get_loc('time(s)')
df_right = df_clean.iloc[:, index+1:]
X = df_right
Y = df_clean['Exercise']

In [5]:
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

In [6]:
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(Y_encoded, dtype=torch.long)

In [7]:
# Assuming X is your features and y is your labels
X_train, X_temp, y_train, y_temp = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)

# Split the temp set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

train_dataset = TensorDataset(X_train,y_train)
test_dataset = TensorDataset(X_test,y_test)
trainLoader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testLoader = DataLoader(test_dataset, batch_size=32 , shuffle=True)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=X_train.shape[1], hidden_size1=64,hidden_size2=64, output_size=7):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3=nn.Linear(hidden_size2, output_size)

        self.reset_parameters()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x= self.relu(x)
        x=self.layer3(x)
        return x
        
    def reset_parameters(self):
        for layer in self.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()



In [ ]:
class GRUNeuralNetwork(nn.Module):
    def __init__(self):
        super(GRUNeuralNetwork, self).__init__()
        self.gru = nn.GRU(X_train.shape[1], 64)#, batch_first=True)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 7)

    def forward(self, x):
        # x est de la forme (batch_size, sequence_length, input_size)
        output, _ = self.gru(x)
        # Prenez seulement la sortie de la dernière étape de la séquence
        #output = output[:, -1, :]
        output = self.relu(output)
        output = self.layer2(output)
        return output

**train**

In [9]:
model= NeuralNetwork()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [12]:
model.train()

for epoch in range(4):  # 4 is the nbr of epochs
    running_loss = 0.0
    total_train = 0
    correct_train = 0

    for inputs, labels in trainLoader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculer la perte moyenne et l'exactitude pour l'époque
    epoch_loss = running_loss / len(trainLoader)
    epoch_accuracy = 100 * correct_train / total_train
    print(f'Époque {epoch + 1}, Perte: {epoch_loss:.4f}, Exactitude: {epoch_accuracy:.2f}%')

Époque 1, Perte: 0.2622, Exactitude: 90.15%
Époque 2, Perte: 0.1922, Exactitude: 92.17%


KeyboardInterrupt: 

**hyper parmeters tuning**
- nb of neurons layers 1 and 2 
- batch_size
- learning rate 

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from skorch.callbacks import EarlyStopping
from skorch import NeuralNetClassifier

In [14]:
def tune_hyperparameters(X_valid, y_valid):
    #parameters to tune
    param_grid = {
        'module__hidden_size1': [32, 64, 128],
        'module__hidden_size2': [32, 64, 128],
        'batch_size': [10, 20, 40, 60, 80, 100],
        'optimizer__lr': [0.001, 0.01, 0.1, 0.2, 0.3]
    }

    model = NeuralNetwork(input_size=X_valid.shape[1], hidden_size1=64, hidden_size2=64, output_size=7)

    train_dataset = TensorDataset(X_valid, y_valid)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    early_stopping = EarlyStopping(
        monitor='valid_loss',  # Change to 'valid_acc' for accuracy
        threshold=0.01,       # Define your threshold
        threshold_mode='rel',  # 'rel' for relative, 'abs' for absolute
        patience=5            # Number of epochs to wait after condition is met
    )


    # Convert the PyTorch model to a skorch classifier to use in GridSearchCV
    classifier = NeuralNetClassifier(
        model,
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.Adam,
        max_epochs=10, # or choose an appropriate number of epochs
        callbacks=[early_stopping] 
    )

    # Use GridSearchCV for hyperparameter tuning, cv for the number of folds in cross-validation, verbose for the explicit stage of tuning
    grid_search = GridSearchCV(classifier, param_grid, scoring='accuracy', cv=3, verbose=1)
    #get grid result
    grid_result= grid_search.fit(X_valid, y_valid)

    # Get the best hyperparameters
    best_hyperparams = grid_search.best_params_

    #get best score
    best_score=grid_search.best_score_

    return best_hyperparams, best_score


In [15]:
model.eval()
#tune 
best_params, best_score = tune_hyperparameters(X_val, y_val)
print(best_params)

Fitting 3 folds for each of 90 candidates, totalling 270 fits
  epoch    train_loss    valid_acc    valid_loss       dur
-------  ------------  -----------  ------------  --------
      1        0.2208       0.9234        0.1879  201.0586
      2        0.1829       0.9274        0.1770  172.7593
